In [3]:
import numpy as np
import random
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
# add project root folder to path to allow import local modules
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
# import local modules
from stochastic_models import *
from visualisations import *

In [4]:
np.random.seed(2020)

## Model Parameters

In [5]:
N_train = 2000
N_test = 10000
d = 1
T = 2


In [6]:
lambda_range = (N_train*1e-9 , N_train*1e-3)
alpha_range = (8.3*1e-5, 0.83)
length_scale = np.sort(1/np.sqrt((2*alpha_range[0], 2*alpha_range[1])))

In [7]:
def reconstruct_alpha(length_scale):
    return 1/(2*(length_scale**2))
def reconstruct_lambda(noise_level, N):
    return noise_level/N

## Kernel Construction

In [8]:
#Kernel Construction, doubt on the lambda hyperparameter
kernel = RBF(length_scale= (length_scale[0] + length_scale[1])/2, length_scale_bounds=length_scale) \
        + WhiteKernel(noise_level= (lambda_range[0] + lambda_range[1])/2 , noise_level_bounds=lambda_range)


## Data Generation (Test, Train sets)

In [9]:
s_train = MaxCallStochasticModel(N_train,d,[1/12,11/12])
s_train.generate_samples()
s_test = MaxCallStochasticModel(N_test, d, [1/12,11/12])
s_test.generate_samples()

In [10]:
y_train = s_train.y
X_train = s_train.X
S_train = s_train.S



In [11]:
y_test = s_test.y
X_test = s_test.X
S_test = s_test.S

V_T = s_test.generate_true_V(T)
V_0 = s_test.generate_true_V(0)

V_0= V_0.mean()
V_0 # Average expected PROFIT!

0.07892604992843537

## Data Flattening 

In [12]:
def Flatten_Training_Sample(X , f):
    return np.array([f(x) for x in X])

In [13]:
#Shape of each sample j: [X_j[0,0],... ,X_j[d,0],...,X_j[0,T] , ..., X_j[d,T]]
Flatten_X_1_train = Flatten_Training_Sample(X_train, lambda x : x.T.flatten())
Flatten_X_1_test = Flatten_Training_Sample(X_test, lambda x : x.T.flatten())


#Shape of each sample j: [X_j[0,0],... ,X_j[0,T],...,X_j[d,0] , ..., X_j[d,T]]
Flatten_X_2_train = Flatten_Training_Sample(X_train, lambda x : x.flatten())
Flatten_X_2_test = Flatten_Training_Sample(X_test, lambda x : x.flatten())

In [14]:
Flatten_X_1_train.shape

(2000, 2)

In [15]:
Flatten_X_1_test.shape

(10000, 2)

## Model Fitting

In [20]:
#Model with Flatten_X_1
m_1 = GaussianProcessRegressor(kernel)
m_1.fit(Flatten_X_1_train,y_train)
fX_1 = m_1.predict(Flatten_X_1_test)

In [21]:
#Model with Flatten_X_2
m_2 = GaussianProcessRegressor(kernel)
m_2.fit(Flatten_X_2_train, y_train)
fX_2 = m_2.predict(Flatten_X_2_test)

## Optimal Hyperparameters

In [ ]:
optimal_hyperparameters_1 = (reconstruct_alpha(np.exp(m_1.kernel.theta[0])),reconstruct_lambda(np.exp(m_1.kernel.theta[1]), N_train))
optimal_hyperparameters_1

In [ ]:
optimal_hyperparameters_2 = (reconstruct_alpha(np.exp(m_2.kernel.theta[0])), reconstruct_lambda(np.exp(m_2.kernel.theta[1]), N_train))
optimal_hyperparameters_2

## Error Calculation

In [25]:
Normalized_Error_T_1 = np.sqrt(1/N_test*np.sum((fX_1-V_T)**2, axis=0))/V_0
## see formula p4 for ||f(X)||2,Q 
## since samples are drawn according to measure - just sum them up

In [26]:
Normalized_Error_T_1

0.165940082833466

In [27]:
Normalized_Error_T_2 = np.sqrt(1/N_test*np.sum((fX_2-V_T)**2, axis=0))/V_0

In [28]:
Normalized_Error_T_2

0.165940082833466